저상버스의 시간표를 저장하는 DB

In [1]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy.sql import select

In [2]:
engine = create_engine("sqlite:///", echo=False)
metadata = MetaData()

Timetable = Table('Timetable', metadata, 
              Column('_id', Integer, primary_key =True, autoincrement=True ),
              Column('routeId', Integer, nullable = False),
              Column('date', String(8), nullable = False), 
              Column('arrivalTm',String(6)),
              Column('posX',String), 
              Column('posY',String))  

metadata.create_all(engine) 
conn = engine.connect()

버스위치api를 반복 호출하면서 현재 운행중인 저상버스가 정류소에 도착하는 시간을 DB에 저장
- 호출했을 때의 위치정보 제공 시간이 이전 호출했을 때의 제공 시간과 다르면서 정류장에 도착해있을 때(stopflag=1) 저장
- 여러 서비스키를 사용하여 하나의 키의 traffic을 모두 사용했을 때 다른 서비스 키 사용


In [3]:
import requests
from bs4 import BeautifulSoup
import time

In [4]:
serviceKey = [
'17fE4NH591MbdnqzN321LiGIFYgJ6XG3LEwBDNDeVYFQm6B9cG24yEdGLjpUa0zu7L1uJagT1fAlxXl9YlSd7Q%3D%3D',
'xsYCorXQLS6GeIHlSblN8cDUql3lya%2FhmRun3ZWAg4uwyPSPZX1ymJcPzt9yDNco1RdLO0Q28d542JfeI%2FoGRQ%3D%3D',
'pluGlBNSzwcQD9L6%2Fq%2B37%2F7ZZOYcjonWJ7eeqjUPy8dmhx9NxyTeEHPsmaXOekWlvImkztRMjZDI4Asp2%2BXdyA%3D%3D',
'ydnnpHpT6l2Y4DExg3mptI1GVHORb0OrGUz%2FlWFNg1rTa5OdqRKmHtfct8SF%2B84E84ZNRev03220ZCpElCzKGQ%3D%3D',
'fAVoRthJ7w6qLml%2Bs%2FVn2z6uXx5wba3vEIUZtl8S73v30WXT0RoQTJmOyDa6BhmRCxO3vaI0UJkOwZdIrosY7g%3D%3D',
'7tEdncbTNM%2B53QBBwElYZLgu1wPpxc3oZn%2FMIceVEB%2Fn3MPrYbIoDPlcsYLPAcCnNQpSTMq7A%2FMM%2BSycwtDm8Q%3D%3D',
'WKji4TSJk6O7Q8LBRLJma0%2BpGuGuGwC5%2FHYgacryM4AzLtHo8zccch60szLmam7YBLzPPPfSKBfVu9hNXFEYGA%3D%3D',
'eFOmgSlNbqagW9tpdkIXv0FdPqQCAFGzhPLp8zsJarW%2F%2F8KXxHsGDI0WPPnFz2jFSKDc7o6pkLDavJyAVhBQjg%3D%3D',
'pvRKIPC0dicOxZ%2FvMNIO%2FpvuiLdq5B9ugO3XnBVBBthwwZuD20IS19afpNhX94MPX1pvE4rGKewrbbvXqO6yeQ%3D%3D',
'zDf1VAbwhMoKbU3Tn%2FpUK5S7H7c3RdOU%2B%2FpReOEFBw24pImKGWejUvXCTJebWwXfp%2Byj95i7mF0oIFDFc5shfA%3D%3D',
'LItL5w1RWPukBFoiKQ%2FD7PpMoiPbmUdbIqzEHCMFaTuCWjul4w1D52XgsX0V58ZedNFdmOKFtiPT%2FWEHdiT5uw%3D%3D',
'Gd%2BMq2gtJ5b9dmlPh6r6hg0j6zXCN9dndZd%2FM3S7D0TqyXlRV%2FEyOX54yF7GTK4jms31lMokRk4vZy48ULCr6A%3D%3D',
'4fxQgZUy2hbO%2FEU61rpuNySpc3%2BGWZVXnRw0lkd0wireOAdSTyLaiErL%2FVOBnP5YTe5nXRJdFv3FyMcH%2Bh6lbg%3D%3D',
'%2FGN%2BQBieLa7gj0y5bGz9N7%2B%2BYY5J3T2VxMQ9K7nOF8mwLAabgyvcco7fYBW5o%2BHUIs98srxPiD%2FylJ0h5JqYtg%3D%3D',
'ebhoaoKgsYhoOWNmEmqqaPWV9rKLHJKfa6FCMfBFOfOh4%2B%2FEqp3P0GxwYxSdd6wHklE0OaNxrR0BPCLMgO4GWQ%3D%3D', 
'zzIkK5JvK5Qf0PQlG%2BGen30%2B74nnNDH%2BHUa8Gj21q0astwZNWgcy8Ul7O34wKGYUL3tgVelscNe0T1ngsr8LhA%3D%3D',
'T7e%2BPYf146tuYKxEcZDTv9CowyQ5IG6bt0vaig3oBkGyMYzOMMj43DA7NS2Kg6paIZr46vrlAANG0MYfSAG6ug%3D%3D', 
'A6hnPJg5aBy29z%2BQw%2B7O5qsGHwriJTM2tkdTnWqRK2GsBXBi7nPEg%2BlgO2kxTVbuEH5GWCijnDlmFYrjYtxJ8A%3D%3D',
'JQOuA6H%2B5GED2ebN1y3EeDRkQwr9BQczLJBUPE%2FHnD13IJJiWXLtRYd5PEtl%2FBqBUfojSzwyYk8SCvdIVcmYGQ%3D%3D',
'mhHYcau5uvPAk485nkwUxwPmSwHBOzgAvN5M8WYMsRLTBhbV8plJSMQcTXOJ4qelyWqJZZjLbSbx9hLDrt8GGg%3D%3D',
'9JY1jEJr%2Fo411aMc1WQhEBTR4EL4oEXVEuzo8d8GRMIuLJ9TJwJPj17%2FT%2Fl1i7nVvr%2FM7QDWtGF%2BOukWBs7T6w%3D%3D',
'3XgU8wCs8EYBaH%2BNNfp5FYhtq0Pmm8XbszveY0hANeC35UFCP2R4bblptP14vxU0MLXaMbeicjYca3GTni%2BwXA%3D%3D',
'jinBdygjITTh%2BdbEWG6DajHBUwB2BJbdjVGCpqi3HCAgmqgvZibEVIIz%2BqJ9j0isgQ6MH2IOgoqV5a82x83gZA%3D%3D',
'tCdOLU5Uzt%2FO0KmLIYqZSvUVXMFI09J9Ypf%2FR6zRz7Brg78PyFngcRRiz2%2FLIhmxSir4sJ5fMTlOTsl7kHYrug%3D%3D',
'dn0AsDO%2BDYT4eyDFEIsGiA518Hs%2B%2BKrEeSF9ZbrpYf6A3A2ySnnOCVJBu27ITk%2BdciNwaag4ULUcbSPNZvX3CA%3D%3D',
'xwgODeBQguCeaAZYpZRFwGjBiQJSBlq8XZsN86fBqnZ7msUod8uUfYtjCCDI7HlnpzPHnVlCbN61XHyMmh0VBA%3D%3D',
'liLwK4N1T04lqoN9rQKNX5UniCucaNFEWdDSjJL8lRs7veBVNLBOUczyMNNhVm1prmU4mZFmigmuOiNX9%2B48yg%3D%3D',
'4ehDKWXvirSm7MiRPEeFefnxApsVNXaw3WgxnOoLsOUIXUmZW6hHIFbgmkaYOUxK915rN1cTE6FbdkXkml8zjA%3D%3D',
'dNZWlZbeIWGzxUJH0qrd6Tgza4CjxdpdSUvbVsbWDsv1FkD1dV7J4ziLqsex40vkVGupL4VaKqqC9cFRM2mBsA%3D%3D',
'1famqleYz%2FAt4bgyjC2qMqqgnJJwYEmmMVYpJ06aoiKQnt1KEgTNQ6yW1lAs2QvIRsc3iT8IU7k9b32%2F8LyD4w%3D%3D',
"ifKRU73xKAdgp5Z2cQ6BDZSHhEOvKL8TNiEegFTFxtM9AWL6l0Wx8WJ4DfmQDZrTjjb509tqW0hO2wA9FE%2BWsQ%3D%3D"
]

In [5]:
len(serviceKey)

31

하나의 키로 하나의 버스 노선의 현재 운행중인 버스 목록을 가져오는 함수

In [6]:
def getData(key, routeID):
    url =  "http://ws.bus.go.kr/api/rest/buspos/getLowBusPosByRtid"
    data = requests.get(url+ "?serviceKey="+key+"&busRouteId="+routeID)
    xml = BeautifulSoup(data.text, "lxml")
    busList = xml.findAll("itemlist") #해당 노선의 현재 운행중인 모든 저상버스
    
    return busList


In [7]:
lowbusID = ['100100570', '100100549', '100100006', '100100007', '100100010', '100100012', '100100016', '100100015', '100100017', '100100583', '100100018', '100100019', '100100020', '100100021', '100100022', '100100023', '100100024', '100100025', '100100026', '100100027', '100100029', '100100030', '100100031', '100100032', '100100033', '100100034', '100100036', '110000002', '100100038', '100100039', '100100407', '100100042', '100100043', '100100044', '100100046', '100100047', '100100048', '100100049', '100100051', '100100052', '100100053', '100100564', '100100496', '100100055', '100100056', '124000038', '100100553', '100100057', '124000036', '100100061', '100100596', '100100062', '100100063', '100100597', '100100064', '100100461', '100100068', '100100409', '100100459', '100100070', '113000002', '100100071', '100100605', '100100073', '100100074', '100100075', '100100076', '100100077', '100100410', '100100078', '100100079', '100100080', '100100081', '100100082', '100100083', '100100574', '100100084', '100100085', '100100086', '100100087', '100100088', '100100090', '100100091', '100100093', '100100094', '100100096', '100100097', '100100098', '100100497', '115000007', '100100102', '100100550', '100100436', '100100495', '100100551', '100100440', '100100103', '100100104', '100100105', '100100107', '100100587', '100100511', '100100110', '100100111', '100100112', '100100537', '100100114', '100100565', '100100116', '100100117', '100100118', '111000014', '100100437', '111000011', '100100131', '100100143', '100100144', '100100148', '100100154', '100100155', '100100159', '100100164', '100100165', '100100175', '100100177', '100100179', '100100181', '100100184', '100100185', '100100186', '100100188', '100100189', '100100190', '100100522', '100100198', '100100199', '100100202', '100100203', '100100204', '100100205', '100100599', '100100611', '100100209', '100100210', '100100211', '100100438', '100100579', '100100213', '100100215', '100100216', '100100218', '100100220', '100100221', '100100222', '100100478', '100100498', '100100501', '100100578', '100100613', '100100223', '100100224', '100100225', '100100226', '100100228', '100100229', '100100232', '100100609', '100100612', '100100234', '100100427', '100100500', '100100454', '100100246', '122000001', '100100248', '100100254', '100100257', '100100258', '100100259', '100100260', '100100261', '100100263', '100100264', '100100265', '100100266', '100100267', '100100268', '100100269', '100100272', '100100273', '100100274', '100100275', '100100276', '100100277', '100100279', '100100282', '100100283', '100100284', '100100285', '100100288', '100100289', '100100290', '100100291', '100100292', '100100294', '100100521', '100100576', '100100300', '100100301', '100100304', '100100305', '100100306', '100100307', '100100308', '100100311', '100100313', '100100601', '100100602', '100100318', '115000005', '100100329', '100100330', '100100331', '100100332', '100100451', '100100449', '100100448', '100100447', '100100337', '100100338', '100100339', '100100340', '100100341', '100100446', '100100344', '100100499', '100100345', '100100346', '100100347', '100100348', '100100462', '100100352', '100100353', '100100360', '111000010', '100100001', '100100003', '165000381', '222000032', '207000004', '213000011', '222000045', '235000020', '234000040', '210000002', '229000041', '222000028', '222000048', '222000058', '232000095', '227000006', '227000001', '234000003', '207000027', '210000013', '210000003', '210000001', '204000012', '224000008', '210000009', '204000005', '210000069', '210000042', '204000013', '210000037', '207000002', '236000048', '233000031', '200000149', '210000093', '222000084', '212000002', '222000008', '222000111', '222000027', '222000009']

In [8]:
len(lowbusID)

286

각각의 버스노선을 객체로 만들어 현재 운행중인 해당 노선의 버스 목록과 각 버스의 위치정보 제공시간을 저장함

In [9]:
class LowBus:
    def __init__(self, routeID):
        self.id = routeID
        self.busList = {}
        self.bus_time = {}

In [10]:
lowbusList= []

for lowbus in lowbusID:
    lowbusList.append(LowBus(lowbus))   #전체 lowbus 객체 생성


for key in serviceKey:

    call =  10000 // len(lowbusID)
    print(key)
#     call = 286
    while call > 0 :  #call 수 만큼 반복 가능 - 할당된 call을 다 사용하면 다음 서비스키 사용
        
        for lowbus in lowbusList:
            print(lowbus.id)
            lowbus.busList = getData(key, lowbus.id)
            call -= 1
            
   
            if lowbus.busList:
                for bus in lowbus.busList:
                    vehid = bus.vehid.text
                    stop = int(bus.stopflag.text)
                    bus_time = bus.datatm.text[8:]
                    print('vehid: ', vehid)
                    print('stop: ', stop)
                    print('time: ', bus_time)
                    print('x, y: ',bus.tmx.text, ', ',bus.tmy.text)
            
                    if vehid in lowbus.bus_time.keys():
                        if lowbus.bus_time[vehid]!=bus_time and stop:  #정류장여부 0->1
                            #DB에 시간, 위치 저장
                            conn.execute(Timetable.insert(), routeId = lowbus.id, date = bus.datatm.text[:8], arrivalTm=bus.datatm.text[8:] , posX = bus.tmx.text, posY = bus.tmy.text) 
                            print('db update')
                    elif stop == 1:
                            #DB에 시간 저장
                            conn.execute(Timetable.insert(), routeId = lowbus.id, date = bus.datatm.text[:8], arrivalTm=bus.datatm.text[8:] , posX = bus.tmx.text, posY = bus.tmy.text) 
                            print('db update')
                    lowbus.bus_time[vehid] = bus_time

                    if vehid in lowbus.bus_time.keys() and (int(bus.nextsttm.text) >= int(bus.laststtm.text)):  #종점에 도착한 버스는 pop
                        lowbus.bus_time.pop(vehid)
                
                    print()
                    
            if call <= 0:
                break
                
            print("-----------------------------")      
              
        time.sleep(60)       

        

17fE4NH591MbdnqzN321LiGIFYgJ6XG3LEwBDNDeVYFQm6B9cG24yEdGLjpUa0zu7L1uJagT1fAlxXl9YlSd7Q%3D%3D
100100570
vehid:  122071265
stop:  0
time:  194305
x, y:  126.831882 ,  37.57938

-----------------------------
100100549
vehid:  110056261
stop:  1
time:  194306
x, y:  127.064368 ,  37.645655
db update

vehid:  110056455
stop:  1
time:  194300
x, y:  127.06158 ,  37.63197
db update

vehid:  110056456
stop:  0
time:  194221
x, y:  127.030675 ,  37.609295

vehid:  110056197
stop:  0
time:  194152
x, y:  127.006518 ,  37.588704

vehid:  110056313
stop:  1
time:  194251
x, y:  126.989528 ,  37.566277
db update

vehid:  110056452
stop:  0
time:  194306
x, y:  126.983477 ,  37.56614

vehid:  110056507
stop:  0
time:  194033
x, y:  126.97792 ,  37.561175

vehid:  110056138
stop:  0
time:  194259
x, y:  126.972628 ,  37.5547

vehid:  110056459
stop:  1
time:  194307
x, y:  126.970564 ,  37.53205
db update

vehid:  110056150
stop:  0
time:  194254
x, y:  126.973534 ,  37.519959

vehid:  110056062
stop

time:  194258
x, y:  127.03706 ,  37.588396

vehid:  108026251
stop:  0
time:  194215
x, y:  127.044292 ,  37.571753

vehid:  108026274
stop:  0
time:  194248
x, y:  127.110741 ,  37.543217

vehid:  108026081
stop:  1
time:  194304
x, y:  127.13994 ,  37.537972
db update

vehid:  108026155
stop:  0
time:  194257
x, y:  127.101068 ,  37.546355

vehid:  108026320
stop:  0
time:  194301
x, y:  127.042519 ,  37.572686

vehid:  108026164
stop:  1
time:  194305
x, y:  127.033465 ,  37.601303
db update

vehid:  108026316
stop:  0
time:  194255
x, y:  127.025983 ,  37.638255

vehid:  108026442
stop:  1
time:  194237
x, y:  127.028687 ,  37.640748
db update

vehid:  108026163
stop:  0
time:  194249
x, y:  127.041127 ,  37.659226

-----------------------------
100100019
vehid:  123060100
stop:  0
time:  193726
x, y:  127.043609 ,  37.689718

vehid:  123060708
stop:  0
time:  194305
x, y:  127.045062 ,  37.690283

vehid:  123060539
stop:  1
time:  194251
x, y:  127.040628 ,  37.65874
db update

v

vehid:  110058190
stop:  0
time:  194305
x, y:  127.049021 ,  37.662258

vehid:  110027118
stop:  1
time:  194259
x, y:  127.0467 ,  37.672839
db update

vehid:  110058324
stop:  1
time:  194252
x, y:  127.056826 ,  37.667941
db update

vehid:  110027273
stop:  1
time:  194302
x, y:  127.06283 ,  37.647128
db update

vehid:  110027027
stop:  0
time:  194231
x, y:  127.072089 ,  37.627252

vehid:  110058325
stop:  0
time:  194255
x, y:  127.07874 ,  37.604948

vehid:  110058492
stop:  1
time:  194258
x, y:  127.079665 ,  37.585067
db update

vehid:  110058286
stop:  0
time:  194253
x, y:  127.078483 ,  37.567951

vehid:  110058251
stop:  1
time:  194253
x, y:  127.07275 ,  37.557845
db update

vehid:  110027307
stop:  0
time:  194250
x, y:  127.065818 ,  37.545552

vehid:  110027180
stop:  0
time:  194302
x, y:  127.064118 ,  37.542553

vehid:  110058226
stop:  0
time:  194253
x, y:  127.056501 ,  37.529055

vehid:  110058412
stop:  0
time:  194205
x, y:  127.056705 ,  37.520308

vehid:

vehid:  108018204
stop:  0
time:  194307
x, y:  127.016098 ,  37.620064

vehid:  108018093
stop:  0
time:  194301
x, y:  127.023492 ,  37.607545

vehid:  108018555
stop:  1
time:  194308
x, y:  127.021863 ,  37.580451
db update

vehid:  108018556
stop:  0
time:  194300
x, y:  126.993125 ,  37.566458

vehid:  108018209
stop:  1
time:  194305
x, y:  126.973778 ,  37.558639
db update

vehid:  108018484
stop:  0
time:  194252
x, y:  126.964259 ,  37.525531

vehid:  108018482
stop:  0
time:  194153
x, y:  126.947977 ,  37.513315

vehid:  108018201
stop:  0
time:  194251
x, y:  126.929569 ,  37.484526

vehid:  108018227
stop:  1
time:  194251
x, y:  126.929724 ,  37.48286
db update

vehid:  108018469
stop:  0
time:  194251
x, y:  126.931536 ,  37.478909

vehid:  108018260
stop:  0
time:  194302
x, y:  126.920749 ,  37.452449

vehid:  108018220
stop:  0
time:  194216
x, y:  126.914793 ,  37.432536

vehid:  108018166
stop:  0
time:  194255
x, y:  126.916543 ,  37.444179

vehid:  108018571
stop

vehid:  110054458
stop:  0
time:  194301
x, y:  127.107132 ,  37.64917

vehid:  110054350
stop:  1
time:  194302
x, y:  127.124616 ,  37.65513
db update

vehid:  110054035
stop:  0
time:  194233
x, y:  127.114851 ,  37.646641

vehid:  110054086
stop:  0
time:  194256
x, y:  127.108299 ,  37.638778

vehid:  110054302
stop:  0
time:  194247
x, y:  127.090261 ,  37.623535

vehid:  110054101
stop:  0
time:  194301
x, y:  127.078767 ,  37.604809

vehid:  110054343
stop:  0
time:  194250
x, y:  127.073223 ,  37.593289

vehid:  110054461
stop:  0
time:  194256
x, y:  127.054071 ,  37.588211

vehid:  110054347
stop:  0
time:  194257
x, y:  127.042038 ,  37.579253

vehid:  110054342
stop:  0
time:  194251
x, y:  127.030433 ,  37.577435

vehid:  110054477
stop:  1
time:  194258
x, y:  126.999259 ,  37.566818
db update

vehid:  110054173
stop:  0
time:  194250
x, y:  126.981951 ,  37.563666

vehid:  110054348
stop:  0
time:  194256
x, y:  126.975341 ,  37.559863

vehid:  110054104
stop:  0
time: 

...

vehid:  123060727
stop:  1
time:  194309
x, y:  127.154636 ,  37.497621
db update

vehid:  123060737
stop:  0
time:  194306
x, y:  127.148587 ,  37.501388

vehid:  123060726
stop:  0
time:  194203
x, y:  127.140635 ,  37.518823

vehid:  123036241
stop:  1
time:  194303
x, y:  127.157363 ,  37.491107
db update

-----------------------------
100100478
vehid:  123036317
stop:  1
time:  194310
x, y:  127.09807 ,  37.512871
db update

vehid:  123036316
stop:  0
time:  194310
x, y:  127.155354 ,  37.491439

-----------------------------
100100498
vehid:  123036356
stop:  0
time:  194308
x, y:  127.179122 ,  37.566451

vehid:  124031109
stop:  0
time:  194257
x, y:  127.144248 ,  37.553127

vehid:  124031080
stop:  1
time:  194312
x, y:  127.127694 ,  37.550883
db update

vehid:  124031022
stop:  0
time:  194223
x, y:  127.097994 ,  37.515816

vehid:  124031010
stop:  0
time:  193902
x, y:  127.116559 ,  37.507731

vehid:  124031164
stop:  1
time:  194309
x, y:  127.154401 ,  37.497606
db upd

x, y:  127.0569 ,  37.506533
db update

vehid:  107017175
stop:  1
time:  194312
x, y:  127.061155 ,  37.497937
db update

vehid:  107017292
stop:  1
time:  194317
x, y:  127.079522 ,  37.482565
db update

vehid:  107017178
stop:  0
time:  194227
x, y:  127.089107 ,  37.483863

vehid:  107017167
stop:  1
time:  194305
x, y:  127.104419 ,  37.482528
db update

vehid:  107017125
stop:  1
time:  194304
x, y:  127.12118 ,  37.467915
db update

-----------------------------
100100612
vehid:  123030213
stop:  1
time:  194317
x, y:  127.104674 ,  37.484552
db update

vehid:  123030292
stop:  1
time:  194317
x, y:  127.055015 ,  37.491882
db update

vehid:  123030524
stop:  1
time:  194232
x, y:  127.044135 ,  37.509851
db update

vehid:  123030066
stop:  1
time:  194314
x, y:  127.042805 ,  37.51763
db update

vehid:  123030586
stop:  0
time:  194301
x, y:  127.049661 ,  37.502862

vehid:  123030217
stop:  0
time:  194311
x, y:  127.063222 ,  37.494383

vehid:  123030417
stop:  0
time:  19423

db update

-----------------------------
100100260
vehid:  114070432
stop:  0
time:  193945
x, y:  126.919291 ,  37.464127

vehid:  114070342
stop:  1
time:  194229
x, y:  126.919522 ,  37.472257
db update

vehid:  114070358
stop:  0
time:  194312
x, y:  126.913559 ,  37.486701

vehid:  114070357
stop:  0
time:  194313
x, y:  126.908598 ,  37.485048

vehid:  114070366
stop:  1
time:  194306
x, y:  126.927575 ,  37.489367
db update

vehid:  114070412
stop:  1
time:  194309
x, y:  126.954745 ,  37.485844
db update

vehid:  114070368
stop:  0
time:  194302
x, y:  126.963455 ,  37.476968

vehid:  114070367
stop:  1
time:  194302
x, y:  126.982072 ,  37.48245
db update

vehid:  114070411
stop:  0
time:  194247
x, y:  126.977843 ,  37.502281

vehid:  114070370
stop:  1
time:  194257
x, y:  126.96311 ,  37.508184
db update

vehid:  114070379
stop:  0
time:  194312
x, y:  126.98343 ,  37.500584

vehid:  114070380
stop:  0
time:  194258
x, y:  126.982035 ,  37.486921

vehid:  114070442
stop:  1

x, y:  126.927483 ,  37.515384

vehid:  116005372
stop:  1
time:  194311
x, y:  126.923253 ,  37.523355
db update

vehid:  116005304
stop:  1
time:  194312
x, y:  126.927058 ,  37.523599
db update

vehid:  116005182
stop:  0
time:  194301
x, y:  126.924264 ,  37.521829

vehid:  116005067
stop:  1
time:  194241
x, y:  126.916607 ,  37.496986
db update

vehid:  116005191
stop:  0
time:  194311
x, y:  126.902982 ,  37.484984

vehid:  116043441
stop:  0
time:  194300
x, y:  126.897719 ,  37.467894

vehid:  116043443
stop:  1
time:  194313
x, y:  126.897898 ,  37.470015
db update

vehid:  116043449
stop:  0
time:  194221
x, y:  126.901098 ,  37.452921

vehid:  116005137
stop:  1
time:  194309
x, y:  126.905883 ,  37.422409
db update

vehid:  116005129
stop:  1
time:  194250
x, y:  126.916524 ,  37.404395
db update

vehid:  116005258
stop:  0
time:  194244
x, y:  126.92294 ,  37.361119

-----------------------------
100100282
vehid:  116043171
stop:  0
time:  194302
x, y:  126.949978 ,  37.4

KeyboardInterrupt: 

In [11]:
for row in conn.execute(select([Timetable])):
    print(row)

(1, 100100549, '20180817', '194306', '127.064368', '37.645655')
(2, 100100549, '20180817', '194300', '127.06158', '37.63197')
(3, 100100549, '20180817', '194251', '126.989528', '37.566277')
(4, 100100549, '20180817', '194307', '126.970564', '37.53205')
(5, 100100549, '20180817', '194302', '126.966137', '37.527272')
(6, 100100549, '20180817', '194255', '127.000949', '37.566721')
(7, 100100549, '20180817', '194254', '127.017925', '37.593414')
(8, 100100006, '20180817', '194144', '127.034913', '37.658003')
(9, 100100006, '20180817', '194259', '127.026139', '37.638613')
(10, 100100006, '20180817', '194301', '127.031808', '37.585267')
(11, 100100006, '20180817', '194259', '126.971434', '37.570001')
(12, 100100006, '20180817', '194249', '127.034036', '37.647307')
(13, 100100007, '20180817', '194252', '127.021782', '37.597733')
(14, 100100007, '20180817', '194248', '127.062793', '37.647972')
(15, 100100010, '20180817', '194302', '127.04109', '37.578951')
(16, 100100010, '20180817', '194248', 